In [1]:
!gdown  18cij6foP2ilw2NtadINSnYP6UUbenXCf

Downloading...
From: https://drive.google.com/uc?id=18cij6foP2ilw2NtadINSnYP6UUbenXCf
To: /content/scc5966.zip
100% 18.4M/18.4M [00:00<00:00, 105MB/s] 


In [2]:
!unzip scc5966.zip

Archive:  scc5966.zip
  inflating: movie_reviews.csv       
  inflating: movies_data.csv         
  inflating: test_data.csv           
  inflating: train_data .csv         
  inflating: users_data.csv          


In [3]:
import pandas as pd 

df_movie_reviews = pd.read_csv('movie_reviews.csv')

df_movies_data = pd.read_csv('movies_data.csv')

df_test_data = pd.read_csv('test_data.csv')

df_train_data = pd.read_csv('train_data .csv')

df_users_data = pd.read_csv('users_data.csv')

In [4]:
def pred(df, const, user, item):

  bias_item = 0
  if item in df.columns:
    item_len = len(df[item][~df[item].isnull()])
    for rating_user_item in df[item][~df[item].isnull()]:
      bias_item = bias_item + (rating_user_item - global_avg)
  else:
    item_len = 0
  
  bias_item = bias_item/(const + item_len)

  bias_user = 0
  if user in df.index:
    user_len = len(df.loc[user][~df.loc[user].isnull()])
    for rating_user_item in df.loc[user][~df.loc[user].isnull()]:
      bias_user = bias_user + (rating_user_item - global_avg - bias_item)
  else:
    user_len = 0

  bias_user = bias_user/(const + user_len)

  rating = global_avg + bias_item + bias_user
  
  return rating
  
def pred_generic(parameters):
  return pred(parameters[0], parameters[1], parameters[2], parameters[3])
  
def generate(parameters):
  l= []
  
  for item,row in tqdm(df_test_data.iterrows()):
    user = row['user_id']
    item = row['movie_id']
    if user not in df_train_data['user_id'].unique() and item in df_train_data['movie_id'].unique():
      l.append(items_avg[item])
    elif user in df_train_data['user_id'].unique() and item not in df_train_data['movie_id'].unique():
      l.append(user_avg[user])
    elif user not in df_train_data['user_id'].unique() and item not in df_train_data['movie_id'].unique():
      l.append(global_avg)
    else:
      parameters.append(user)
      parameters.append(item)

      value = pred_generic(parameters)
      if value < 1:
        value = 1
      if value > 5:
        value = 5

      l.append(value)
  
  return l


In [7]:
import time
import pandas as pd
import numpy as np
from tqdm import tqdm

df_user_item = pd.DataFrame(index = df_train_data['user_id'].unique(), columns=df_train_data['movie_id'].unique())

for movie in df_train_data['movie_id'].unique():
  temp = df_train_data[df_train_data.movie_id == movie]
  df_user_item.loc[temp.user_id, movie] = temp.rating.to_list()
  
items_avg = np.mean(df_user_item)
user_avg = np.mean(df_user_item.T)
global_avg = np.nanmean(df_user_item.values.tolist())

In [8]:
const = 0.0000000000001

y_pred = generate([df_user_item, const])

df_test_data['rating'] = y_pred
df_rating = df_test_data[['id','rating']]
df_rating = df_rating.set_index('id')
name = 'FC_Baseline_' + str(const) + '.csv'
df_rating.to_csv('/content/drive/MyDrive/USP/Doctorate/Credits/Recomender_Systems/results-kaggle/' +name)

3970it [01:29, 44.59it/s]
